<i>Notebook jupyter python for automatic extraction of information in semi-structured files using mainly regular expressions; then format them and store them in other files which will be used to populate an sql database </i>

In [263]:
#loading librairies
import pandas as pd
import numpy as np
import re
from datetime import datetime
import locale
import calendar
import os
from tqdm import tqdm

# Helpers Variables definition

<p>Define all the variables pattern and regex we'll for future computations</p>

In [ ]:
global types_marche
types_marche = ['.*REPARTITION DES CREDITS PAR NATURE ET PAR DUREE$',
                '.*REPARTITION DES CREDITS PAR TYPE DE CLIENTELE$',
                '.*SERIES HISTORIQUES DES DEPOTS$',
                '.*REPARTITION GEOGRAPHIQUE DES CREDITS PAR VILLE$',
                '.*REPARTITION GEOGRAPHIQUE DES CREDITS PAR REGION$',
                '.*REPARTITION DES DEPOTS PAR NATURE$',
                '.*REPARTITION DES DEPOTS PAR TYPE DE CLIENTELE$',
                '.*REPARTITION GEOGRAPHIQUE DES DEPOTS$',
                '.*REPARTITION GEOGRAPHIQUE DES DEPOTS PAR VILLE$',
                '.*REPARTITION GEOGRAPHIQUE DES DEPOTS PAR REGION$',
                '.*REPARTITION DES CREDITS PAR DUREE ET TYPE DE CLIENTELE$',
                '.*REPARTITION DES DEPOTS PUBLICS PAR DUREE ET TYPE DE CLIENTELE$',
                '.*REPARTITION DES CREDITS PUBLICS PAR DUREE ET TYPE DE CLIENTELE$',
                '.*SERIES HISTORIQUES DES CREDITS$'
               ]

comb_type = join_pattern(types_marche)


columns = {
    'REPARTITION DES CREDITS PAR NATURE ET PAR DUREE':['Montant', 'Banque', 'Nature', 'Type_Nature'],
    'REPARTITION DES CREDITS PAR TYPE DE CLIENTELE':['Montant', 'Banque', 'Nature', 'Type_Nature'],
    'SERIES HISTORIQUES DES DEPOTS':['Montant', 'Date_série', 'Banque', 'Type_Nature'],
    'REPARTITION GEOGRAPHIQUE DES CREDITS PAR VILLE':['Nbre comptes', 'Montant', 'ville', 'Banque'],
    'REPARTITION GEOGRAPHIQUE DES CREDITS PAR REGION':['Nbre comptes', 'Montant', 'Région', 'Banque'],
    'REPARTITION DES DEPOTS PAR NATURE':['Montant', 'Banque', 'Nature', 'Type_Nature'],
    'REPARTITION DES DEPOTS PAR TYPE DE CLIENTELE':['Montant', 'Banque', 'Nature', 'Type_Nature'],
    'REPARTITION GEOGRAPHIQUE DES DEPOTS': ['Nbre comptes', 'Montant', 'ville', 'Banque'],
    'REPARTITION GEOGRAPHIQUE DES DEPOTS PAR VILLE':['Nbre comptes', 'Montant', 'ville', 'Banque'],
    'REPARTITION GEOGRAPHIQUE DES DEPOTS PAR REGION':['Nbre comptes', 'Montant', 'Région', 'Banque'],
    'REPARTITION DES CREDITS PAR DUREE ET TYPE DE CLIENTELE': ['Montant', 'Banque', 'Nature', 'Type_Nature'],
    'REPARTITION DES DEPOTS PUBLICS PAR DUREE ET TYPE DE CLIENTELE':['Montant', 'Banque', 'Nature', 'Type_Nature'],
    'REPARTITION DES CREDITS PUBLICS PAR DUREE ET TYPE DE CLIENTELE':['Montant', 'Banque', 'Nature', 'Type_Nature'],
    'SERIES HISTORIQUES DES CREDITS':['Montant', 'Date_série', 'Banque', 'Type_Nature']
}


"""
    Patterns defintions to match all possibilities
    and make the right action 
"""

# join patterns funxtion
def join_pattern(patterns):
    return "(" + ")|(".join(patterns) + ")"

# Nature patterns
nature_patterns = ['CREDITS A LONG TERME', 'CREDITS A MOYEN TERME', 'CREANCES BRUTES EN SOUFFRANCE', 
                   'COMPTES DE DEPOTS A REGIME SPECIAL', 'COMPTES CREDITEURS A VUE', '.*DEPOTS VENTILABLES.*',
                  'ADAMAOUA/NORD/EXTR. NORD', 'CENTRE/SUD/EST', 'LITTORAL', 'NORD-OUEST/SUD-OUEST', 'OUEST','ADMINISTRATION PUBLIQUE CENTRALE',
                   'ADMINISTRATION PUBLIQUE LOCALE', 'ORGANISMES PUBLICS', 'ENTREPRISES PUBLIQUES', 'SOCIETES D\'ASSURANCE ET CAPITAL'
                   'ENTREPRISES INDIVIDUELLES', 'PARTICULIERS'
                  ]
comb_nature = join_pattern(nature_patterns)

# Ignore patterns
patterns_ignore_fields = ['.*TOTAL.*', '.*PARTS DE MARCHE.*']
comb_ignore = join_pattern(patterns_ignore_fields)

# Taked into account inside Nature
nature_skipped_patterns = ['COMPTES DEBITEURS DE LA CLIENTELE', 'AUTRES SOMMES DUES PAR LA CLIENTELE', 'PROVISIONS CONSTITUEES']
comb_skip = join_pattern(nature_skipped_patterns)


# Tables to avoid
excludes_tables = ['.*POURCENTAGE.*', 'PART DE REFINANCEMENT', 'RATIO DE TRANSFORMATION', 
                   'INTERMEDIATION', '.*SOUFFRANCE.*']
comb_exclude = join_pattern(excludes_tables)


# Reprocess names:
process_names = ['.*- à vue.*', '.*- à terme.*', '.*- Crédits à court terme.*', '.*- Crédits à long terme.*', '.*- Crédits à moyen terme.*', '.*- Cptes débiteurs clientèle.*',
                '.*- Autres sommes dues par la clientèle.*', '.*A - ADMINISTRATION PUBLIQUE CENTRALE.*', '.* B - ADMINISTRATION PUBLIQUE LOCALE.*',
                 '.*C - ORGANISMES PUBLICS.*', '.* D - ENTREPRISES PUBLIQUES.*', '.*- à moyen et long terme.*', '.*- à court terme.*'
                ]
com_process_names = join_pattern(process_names)

# Functions definition

<p> Define the helper functions regrouping all functionnalities </p>

## 1- Get indices

In [2]:
#Initialize dict indices
#global indices

indices = dict()
for typ in types_marche:
    indices[typ] = list()

# Get all tables indices
def get_indices(datas):
    
    matchs_geo = ['GEOGRAPHIQUE', 'VILLE']
    indices = []
    
    for i in range(len(datas)):
        
        value = datas[i][0]
        if not isinstance(value, str):
            continue

        
        value = value.split(':')
            
        if len(value) != 2:
            val = value[0]
        else:
            val = value[1]
            
        final_val = " ".join(val.split())
        
        
        if re.match(comb_type, final_val):
            
            typ = None
                        
            if matchs_geo[0] in final_val:
                typ = 'region'
                
                if matchs_geo[1] in final_val:
                    typ = 'town'
                    
            indices.append((i, final_val,typ, columns[final_val]))
        else:
            continue
                
                
    return indices

## 2- Get dataframe by type

In [3]:
def get_df_type(datas, start, columns, file_date, type_banc = None):
    
    types = ['region', 'town']
    
    attribute_line = start + 2
    
    datas_values = {}

    list_rows = []
    end = 0
    
    k = start + 4
    
    # Check file type
    if type_banc != None:
        
                
        region_line = datas[attribute_line]
        #region_line = region_line[~np.isnan(region_line)]
        
        #process region:
        final_str=""
        for i in range(region_line.shape[0]):
            if isinstance(region_line[i], str):
                final_str += region_line[i]        
        
        split = final_str.split(':')
        lent = len(split)
        if type_banc == types[0]:
            #print(final_str)
            
            #print((split, lent))
            if lent == 1:
                region = split[0]
            else:
                region = split[1]
            #print(region)
            data_add = ['Pays', region]
        else:
            region = split[1]
            data_add = ['Région', region]
            
        attribute_line += 1
        k +=1

    
    type_nature = None
     
    banks = []
    cur_val = datas[attribute_line]
    
    # get all attributes 
    for j in range(datas[attribute_line].shape[0]):

        if not isinstance(cur_val[j], str):
            continue
        elif re.match(comb_ignore,cur_val[j]) or not isinstance(cur_val[j], str):
            break
        banks.append(cur_val[j])
        
    source_end = '^Source:'
    
    #Compute fields
    while end != 2:
        
        
        #get Nature value
        tab = datas[k][0]
        # Chech first value, and try to match it        
        if not isinstance(tab, str):
            end +=1
            continue
        elif re.match(comb_nature, tab):
            type_nature = tab
            k += 1
            continue
        elif re.match(comb_skip, tab):
            type_nature = None
        elif re.match(comb_ignore,tab):
            k += 1
            continue
            
        if re.match(source_end, tab):
            end = 2
            break
        if re.match(com_process_names, tab):
            tab = tab.split('-')[1]
                
        
        len_attr = len(banks)
        
        if type_banc != None:
            length = len_attr*2
        else:
            length = len_attr
            
        
        vals = datas[k][1:length+1]
        
        # Deal with region and towns
        if type_banc != None:
            vals = [vals[n:n+2] for n in range(0, len(vals), 2)]
            l = [[row[0][0], row[0][1], row[1], tab]  for row in zip(vals, banks)]
        else:
            l = [(list(row)) + [tab, str(type_nature)]  for row in zip(vals, banks)]
            
        #r = [row.append(tab) for row in l]
        list_rows += l
        
        k += 1


    # Process Nature value
    columns = columns
    
    # Process dataframe
    df = pd.DataFrame(list_rows, columns=columns)
    df['Date_file'] = file_date
    if type_banc != None:
        df[data_add[0]]= data_add[1]

    
    return df

## 2 - Files concatenation

In [4]:
def concatenate_files(dict_datas):
    
    final_datas_dict = {}  
    
    print("Concatenation part")
    
    #Process keys
    for key in tqdm(dict_datas):
        
        #process new key
        new_key = key.split()
        new_key = "_".join(new_key)
        
        #save file process
        new_path = "processed_files/" + new_key + ".xlsx"
        concat = pd.concat(dict_datas[key])
        
        #saving
        concat.to_excel(new_path)
        
        #Add in the whole dictionnary
        final_datas_dict[key] = concat
        
        
    return final_datas_dict  

# Main part
<p> Using previous declared and implemented variables / functions </p>

In [ ]:
# set local language (french)
locale.setlocale(locale.LC_ALL, 'fr_FR')

directory = 'datas'

all_datas = {}

for subdir, dirs, files  in os.walk(directory):
    
    for file in tqdm(files):
        
        path = os.path.join(subdir, file)

        #print(path)
        
        #read file
        datas = pd.read_excel(path)
        
        path = path.split('.')
    
        #get corresponded date
        month  = calendar.month_name[int(path[1])]
        date = " ".join([month, path[2]])
        
        
        all_indices = get_indices(datas.values)

        for ind in all_indices:

            #compute dataframe per type
            df = get_df_type(datas.values,ind[0], ind[3], date, ind[2])

            if ind[1] not in all_datas:
                all_datas[ind[1]] = []

            all_datas[ind[1]].append(df)
        
concat = concatenate_files(all_datas)
file  = list(concat.keys())[12]
print(file)
concat[file]

## 2- Process all datas

In [225]:
#Loop over the indices and add dataframe in corresponded key dict
def process(datas, file_date):
    
    all_indices = get_indices(datas.values)

    all_datas = {}
    for ind in all_indices:

        #compute dataframe per type
        df = get_df_type(datas.values,ind[0], ind[3], file_date, ind[2])

        if ind[1] not in all_datas:
            all_datas[ind[1]] = []

        all_datas[ind[1]].append(df)

    return all_datas

In [78]:

#Patterns definition
patterns = ['.*TOTAL.*', 'CREDITS A LONG TERME', 'CREDITS A MOYEN TERME', 'CREANCES BRUTES EN SOUFFRANCE']
computed_fiels = ['.*TOTAL.*', 'PARTS DE MARCHE', '.*DEC.*']

combs = "(" + ")|(".join(patterns[1:]) + ")"

normal_nature = ['COMPTES DEBITEURS DE LA CLIENTELE', 'AUTRES SOMMES DUES PAR LA CLIENTELE', 'PROVISIONS CONSTITUEES']
combs_nature = "(" + ")|(".join(normal_nature) + ")"


# Function definition for types
# REPARTITION DES CREDITS PAR NATURE ET PAR DUREE and REPARTITION DES CREDITS PAR TYPE DE CLIENTELE
def get_type_1(datas, start):
    
     
    attribute_line = start + 4
    type_nature = None
     
    banks = []
    cur_val = datas[attribute_line]
    
    # get all attributes 
    for j in range(datas[attribute_line].shape[0]):

        if not isinstance(cur_val[j], str):
            continue
        elif re.match(patterns[0],cur_val[j]):
            break
        banks.append(cur_val[j])
        
    datas_values = {}

    list_rows = []
    end = 0
    j = start + 4
    
    #Compute fiels
    while end != 2:
        
        #get Nature value
        tab = datas[j][0]
        
        
        # Chech first value, and try to match it        
        if not isinstance(tab, str):
            end +=1
            continue
        elif re.match(combs, tab):
            type_nature = tab
            j += 1
            continue
        elif re.match(combs_nature, tab):
            type_nature = None
        elif re.match(patterns[0],tab):
            j += 1
            continue
        
        
        
        vals = datas[j][1: len(banks)+1]
        
        l = [(list(row)) + [tab, str(type_nature)]  for row in zip(vals, banks)]
        #r = [row.append(tab) for row in l]
        list_rows += l
        
        j += 1
        

    # Process Nature value
    columns = ['Montant', 'Nature', 'Banque', 'Type Nature']
    
    # Process dataframe
    df = pd.DataFrame(list_rows, columns=columns)
    
    return df

# Function definition for type SERIES HISTORIQUES DES CREDITS
get_type_1(datas.values, 85)

,Montant,Nature,Banque,Type Nature
